In [1]:
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import collections
import math
import sys

In [2]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = 16,12

In [3]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD

Using TensorFlow backend.


In [4]:
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

def image_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

def invert(image):
    return 255-image

def display_image(image, color=False):
    if color:
        plt.imshow(image)
    else:
        plt.imshow(image, 'gray')

In [5]:
def resize_region(region):
    return cv2.resize(region, (28, 28), interpolation=cv2.INTER_NEAREST)
def resizeImg(img):
    h, w, ch = img.shape
    height = math.ceil(h/400)
    width = math.ceil(w/900)
    
    resize = 0
    if(width > height):
        resize = width
    else:
        resize = height
        
    resized = cv2.resize(img, None, fx = 1/resize, fy = 1/resize)
    
    return resized

In [11]:
def select_roi(image_orig, image_bin):
    contours, hierarchy = cv2.findContours(image_bin.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    sorted_regions = []
    regions_array = []
    pom = 0
    
    rows, cols, ch = image_orig.shape
    widthMax = math.floor(cols / 15)
    widthMin = widthMax / 3
    heigthMax = 35 * widthMax / 48
    heigthMin = 2 * heigthMax / 3
    
    prevDist = 0
    prevX = 0
    prevY = 0
    minY = rows
    minX = cols
    realY = 0
    false_countours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        if area > heigthMin and h < heigthMax and h > heigthMin and w > widthMin and w < widthMax:
            indeks = 0
            for contour1 in contours:
                x1, y1, w1, h1 = cv2.boundingRect(contour1)
                area = cv2.contourArea(contour1)
                
                if area > heigthMin and h < heigthMax and h > heigthMin and w > widthMin and w < widthMax:
                    if((x < x1 or x + w > x1) and y != y1):
                        pom = 0
                        for false in false_countours:
                            if(false == indeks):
                                pom = 1
                        if(pom == 0):
                            false_countours.append(indeks)
                        break
                    
                indeks = indeks + 1

    
    false_countours.sort(reverse=True) 
    for false in false_countours:
        contours.pop(false)
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        
        if area > heigthMin and h < heigthMax and h > heigthMin and w > widthMin and w < widthMax:
            region = image_bin[y:y+h+1, x:x+w+1]
            regions_array.append([resize_region(region), (x, y, w, h)])
            cv2.rectangle(image_orig, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    regions_array = sorted(regions_array, key=lambda x: x[1][0])
    sorted_regions = [region[0] for region in regions_array]
    return image_orig, sorted_regions

In [7]:
def shearMatrix(pt1, pt2):
    return cv2.getAffineTransform(pt1, pt2)
def shearImg(img_color, src):
    
    dst = cv2.Canny(src, 50, 200, None, 3)
    
    cdst = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
    cdstP = np.copy(cdst)
    
    lines = cv2.HoughLines(dst, 1, np.pi / 180, 150, None, 0, 0)
    
    maxLine = 0
            
    
    
    linesP = cv2.HoughLinesP(dst, 1, np.pi / 180, 50, None, 50, 10)
    l = linesP[0][0]
    
    if linesP is not None:
        for i in range(0, len(linesP)):
            l1 = linesP[i][0]
            lineLength = abs(l1[0] - l1[2])
            if(lineLength > maxLine):
                l = l1
                maxLine = lineLength
    
    cv2.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)
    
    
    
    rows, cols = src.shape
    beginPoint = [l[0], l[3]]
    
    if(l[1] == l[3]):
        beginPoint = [0,0]
    pts1 = np.float32([ beginPoint, [l[0], l[1]], [l[2], l[3]]])
    pts2 = np.float32([ beginPoint, [l[0], l[1]], [l[2], l[1]]])
    
    matrix = shearMatrix(pts1, pts2)
    
    sheared_color = cv2.warpAffine(img_color, matrix, (cols, rows))
    sheared = cv2.warpAffine(src, matrix, (cols, rows))
    
    
    return sheared, sheared_color

In [8]:
def image_bin(image_gs, ind):
    height, width = image_gs.shape[0:2]
    image_binary = np.ndarray((height, width), dtype=np.uint8)
    if(ind == 0):
        image_bin = cv2.adaptiveThreshold(image_gs, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 23, 1)
    else:
        ret, image_bin = cv2.threshold(image_gs, 110, 170, cv2.THRESH_BINARY)
    return image_bin

def dilate(image):
    kernel = np.ones((1, 1))
    return cv2.dilate(image, kernel, iterations=1)

def erode(image):
    kernel = np.ones((2, 2))
    return cv2.erode(image, kernel, iterations=1)

In [12]:
ukupno = 0
sto = 0
dvesto = 0
tristo = 0
ostalo = 0
telefon = 0
for i in range(1, 400):
    extension = '.png'
    if(i >= 400):
        extension = '.jpg'
    img_name = 'img' + str(i) + extension
    image_color = load_image('train-data-set/data-images/' + img_name)
    image_color = resizeImg(image_color)
    img_gray = image_gray(image_color)
    img_shear, image_color = shearImg(image_color, img_gray)
    img = image_bin(img_shear, 1)
    img_bin = dilate(erode(img))
    selected_regions, numbers = select_roi(image_color.copy(), img_bin)
    if(len(numbers) != 12):
        #cv2.imshow("Selected image" + str(i), selected_regions)
        #cv2.waitKey()
        image_color = load_image('train-data-set/data-images/' + img_name)
        image_color = resizeImg(image_color)
        img_gray = image_gray(image_color)
        img_shear, image_color = shearImg(image_color, img_gray)
        img = image_bin(img_shear, 0)
        img_bin = dilate(erode(img))
        selected_regions, numbers = select_roi(image_color.copy(), img_bin)
        if(len(numbers) != 12):
            #cv2.imshow("Fail Selected image" + str(i), selected_regions)
            #cv2.waitKey()
            continue
    if(len(numbers) == 12):
        #cv2.imshow("Selected image" + str(i), selected_regions)
        #cv2.waitKey()
        ukupno = ukupno + 1
        if(i <= 100):
            sto = sto + 1
        elif(i <= 200 and i > 100):
            dvesto = dvesto + 1
        elif(i <= 300 and i > 200):
            tristo = tristo + 1
        elif(i <= 400 and i > 300):
            ostalo = ostalo + 1
        else:
            telefon = telefon + 1
print("Ukupno ", ukupno)
print("Sto ", sto)
print("Dvesto ", dvesto)
print("Tristo ", tristo)
print("Ostalo ", ostalo)
print("Telefon ", telefon)

Ukupno  368
Sto  100
Dvesto  99
Tristo  94
Ostalo  75
Telefon  0
